# Exploration of Collider Bias with discrete Bayes Nets using `pgmpy`

## Collider bias
[Collider bias](https://en.wikipedia.org/wiki/Collider_(statistics) ) can be seen as a form of selection bias. In a causal network, if we have the structure `A -> Collider <- B`, then "conditioning on the collider opens the path between A and B [which can] introduce bias when estimating the causal association between A and B, potentially introducing associations where there are none."

## The `pgmpy` package
Here we use the Python package [pgmpy](https://pgmpy.org) (GitHub repo [here](https://github.com/pgmpy/pgmpy)). There is a useful video of a talk about pgmpy here: https://www.youtube.com/watch?v=DEHqIxX1Kq4. It allows us to specify a Bayesian Net structure, along with the conditional probability distributions. Amongst other things, we can  query the net, conditional upon certain observations.

In [1]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination

## Example 1: Admission to university
This example examines admission to prestigious university which is a function of whether you are good as sports and/or maths. The DAG looks like:

    sports -> admission to prestigious university <- maths
    
Here we have a prior probability of 30% of being good at sports. **Independently**, there is a 30% prior probability of being good at maths. The vector `[0.9, 0.6, 0.6, 0.01]` specifies the probability of being admitted where each column corresponds to:

- sports, maths.  If you are good at sports and maths, there is a high chance of getting admitted
- sports, ¬maths. If you are good at sports but not maths there is am ok chance of getting admitted
- ¬sports, maths. If you are good at sports but not maths there is am ok chance of getting admitted
- ¬sports, ¬maths. If you are not good at sports or maths there a very low chance of getting admitted



In [2]:
model = BayesianModel([('sports', 'admitted'), ('maths', 'admitted')])

# Defining the CPDs:
cpd_a = TabularCPD('sports', 2, 
                   [[0.3, 0.7]],
                   state_names={'sports': ['good', 'bad']})
cpd_b = TabularCPD('maths', 2, [[0.3, 0.7]],
                   state_names={'maths': ['good', 'bad']})
cpd_collider = TabularCPD('admitted', 2, 
                          [[0.9, 0.6, 0.6, 0.01], 
                           [0.1, 0.4, 0.4, 0.99]],
                          evidence=['sports','maths'],
                          evidence_card=[2,2], 
                          state_names={'admitted': ['yes', 'no'], 
                                       'sports': ['good', 'bad'], 
                                       'maths': ['good', 'bad']})

# Associating the CPDs with the network structure.
model.add_cpds(cpd_a, cpd_b, cpd_collider)

# Some other methods
model.get_cpds()

# Check if model is consistent
model.check_model()

True

In [3]:
# Do exact inference using Variable Elimination
infer = VariableElimination(model)

### Calculate P(maths|sports) and P(maths|¬sports)
This is a sanity check. Maths and sports performance are indepdendent in our Bayes Net, and so we should find that P(maths|sports) = P(maths|¬sports).

In [4]:
# Note: sports = 0 is good at sports
result = infer.query(['maths'], evidence={'sports': 'good'})
print("\n\nP(maths | good at sports)")
print(result)  

# Note: sports = 1 is bad at sports
result = infer.query(['maths'], evidence={'sports': 'bad'})
print("\n\nP(maths | bad at sports)")
print(result)

Eliminating: admitted: 100%|██████████| 1/1 [00:00<00:00, 248.20it/s]



P(maths | good at sports)
+-------------+--------------+
| maths       |   phi(maths) |
+=============+==============+
| maths(good) |       0.3000 |
+-------------+--------------+
| maths(bad)  |       0.7000 |
+-------------+--------------+


P(maths | bad at sports)
+-------------+--------------+
| maths       |   phi(maths) |
+=============+==============+
| maths(good) |       0.3000 |
+-------------+--------------+
| maths(bad)  |       0.7000 |
+-------------+--------------+


Confirmed. Results show probability you are talented at maths is independent if you only observe talent in sport.

### Calculate P(maths|sports, admitted) and P(maths|¬sports, admitted)
However, if you only look at those who are admitted (i.e. condition on admission), people who are bad at sports are more likely to be good at maths than those who are bad at maths.

So if you only look at the people who are admitted to university, then you would conclude that 

In [5]:
result = infer.query(['maths'], evidence={'sports': 'good', 'admitted': 'yes'})
print("\n\nP(maths | good at sports, admitted)")
print(result)

result = infer.query(['maths'], evidence={'sports': 'bad', 'admitted': 'yes'})
print("\n\nP(maths | bad at sports, admitted)")
print(result)

Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]



P(maths | good at sports, admitted)
+-------------+--------------+
| maths       |   phi(maths) |
+=============+==============+
| maths(good) |       0.3913 |
+-------------+--------------+
| maths(bad)  |       0.6087 |
+-------------+--------------+


P(maths | bad at sports, admitted)
+-------------+--------------+
| maths       |   phi(maths) |
+=============+==============+
| maths(good) |       0.9626 |
+-------------+--------------+
| maths(bad)  |       0.0374 |
+-------------+--------------+


Top shows there is a ~39% chance of being good at maths if you are good at sports and admitted.

Bottom shows there is a ~96% chance of being good at maths if you are bad at sports and admitted.

So this is our collider bias / selection bias. By looking only at people who are selected for admission, it looks like sports and maths performance are negatively correlated (that you are most likely good at either maths or sports).

# Example 2: Does collider bias make it look like nicotine is protective of COVID

![](img/bayes_net_1.png)

Here we have a prior probability of 25% of being a smoker. **Independently**, there is a 10% prior probability of having Covid-19. The vector `[0.9, 0.1, 0.9, 0.01]` specifies the probability of being in hospital as a function of smoking and covid status:

- smoking, covid. High probability of being in hospital.
- smoking, ¬covid. Moderate probability of being in hospital
- ¬smoking, covid. High probability of being in hospital
- ¬smoking, ¬covid. Low probability of being in hospital

One of the key points here is that of people who do not have covid, smokers are expected to have a higher probability of being in hospital than non-smokers.

In [6]:
model = BayesianModel([('smoking', 'hospital'), ('covid', 'hospital')])

# Defining the CPDs:
cpd_a = TabularCPD('smoking', 2, 
                   [[0.25, 0.75]], 
                   state_names={'smoking': ['yes', 'no']})
cpd_b = TabularCPD('covid', 2, 
                   [[0.1, 0.9]], 
                   state_names={'covid': ['yes', 'no']})
cpd_collider = TabularCPD('hospital', 2, 
                          [[0.9, 0.1, 0.9, 0.01], 
                           [0.1, 0.9, 0.1, 0.99]],
                          evidence=['smoking','covid'],
                          evidence_card=[2,2], 
                          state_names={'hospital': ['yes', 'no'], 
                                       'smoking': ['yes', 'no'], 
                                       'covid': ['yes', 'no']})

# Associating the CPDs with the network structure.
model.add_cpds(cpd_a, cpd_b, cpd_collider)

# Some other methods
model.get_cpds()

[<TabularCPD representing P(smoking:2) at 0x1a23ae31d0>,
 <TabularCPD representing P(covid:2) at 0x1a23ae3250>,
 <TabularCPD representing P(hospital:2 | smoking:2, covid:2) at 0x1a23ae3350>]

In [7]:
# Do exact inference using Variable Elimination
infer = VariableElimination(model)

Probability you have covid depending on if you are a smoker or non-smoker. These should be equal as our Bayes Net specifies that smoking and covid are independent.

In [8]:
result = infer.query(['covid'], evidence={'smoking': 'yes'})
print("\n\nP(covid | smoker)")
print(result)

result = infer.query(['covid'], evidence={'smoking': 'no'})
print("\n\nP(covid | non-smoker)")
print(result)

Eliminating: hospital: 100%|██████████| 1/1 [00:00<00:00, 514.95it/s]



P(covid | smoker)
+------------+--------------+
| covid      |   phi(covid) |
+============+==============+
| covid(yes) |       0.1000 |
+------------+--------------+
| covid(no)  |       0.9000 |
+------------+--------------+


P(covid | non-smoker)
+------------+--------------+
| covid      |   phi(covid) |
+============+==============+
| covid(yes) |       0.1000 |
+------------+--------------+
| covid(no)  |       0.9000 |
+------------+--------------+


Confirmed.

### Calculate P(covid|smoking, hospital) and P(covid|¬smoking, hospital)

If you condition upon being in hospital (ie only observer people who are in hospital), then it might be that we see different probabilities of having covid based on whether you smoke or not.

In [9]:
result = infer.query(['covid'], evidence={'smoking': 'yes', 'hospital': 'yes'})
print("\n\nP(covid | smoker, hospital)")
print(result)

result = infer.query(['covid'], evidence={'smoking': 'no', 'hospital': 'yes'})
print("\n\nP(covid | non-smoker, hospital)")
print(result)

Finding Elimination Order: : : 0it [00:00, ?it/s]
0it [00:00, ?it/s]
Finding Elimination Order: : : 0it [00:00, ?it/s]



P(covid | smoker, hospital)
+------------+--------------+
| covid      |   phi(covid) |
+============+==============+
| covid(yes) |       0.5000 |
+------------+--------------+
| covid(no)  |       0.5000 |
+------------+--------------+



0it [00:00, ?it/s]



P(covid | non-smoker, hospital)
+------------+--------------+
| covid      |   phi(covid) |
+============+==============+
| covid(yes) |       0.9091 |
+------------+--------------+
| covid(no)  |       0.0909 |
+------------+--------------+


Boom! There we have it. There is a _higher_ chance that you will have Covid-19 if you are a _non-smoker_ in hospital, then if you are a smoker in hospital. And this result is produced by a Bayesian Net which explicitly states that having covid is statistically independent from being a smoker.

This is not necessarily proof that nicotine is useless against covid. But it does show that somewhat perplexing epidemiological results should not be taken as sufficient evidence that nicotine is protective against covid.

## Example 3: Slightly more complex Covid/Smoking model

Elaborating the Bayes Net in this way allows us to add in hit rate and false alarm rate 

![](img/model_complex.png)

In [10]:
# Build structure of model
model = BayesianModel([('Covid', 'Symptoms'), 
                       ('Symptoms', 'Hospital'),
                       ('Smoker', 'Hospital'),  
                       ('Covid', 'Positive')])

# Build conditional probability distributions
cpd_c = TabularCPD('Covid', 2, 
                   [[0.1, 0.9]], 
                   state_names={'Covid': ['yes', 'no']})

cpd_s = TabularCPD('Smoker', 2, 
                   [[0.25, 0.75]], 
                   state_names={'Smoker': ['yes', 'no']})

cpd_i = TabularCPD('Symptoms', 2, 
                   [[0.95, 0.001],
                    [0.05, 0.999]], 
                   evidence=['Covid'], 
                   evidence_card=[2], 
                   state_names={'Symptoms': ['yes', 'no'],
                                'Covid': ['yes', 'no']})

cpd_h = TabularCPD('Hospital', 2, 
                   [[0.9, 0.1, 0.9, 0.01], 
                    [0.1, 0.9, 0.1, 0.99]], 
                   evidence=['Smoker', 'Symptoms'], 
                   evidence_card=[2, 2],
                   state_names={'Hospital': ['yes', 'no'],
                                'Smoker': ['yes', 'no'], 
                                'Symptoms': ['yes', 'no']})

hit_rate = 0.95
false_alarm_rate = 0.1
cpd_p = TabularCPD('Positive', 2, 
                   [[hit_rate, false_alarm_rate],
                    [1-hit_rate, 1-false_alarm_rate]], 
                   evidence=['Covid'], 
                   evidence_card=[2], 
                   state_names={'Positive': ['yes', 'no'],
                                'Covid': ['yes', 'no']})

# Add conditional probability distributions to model
model.add_cpds(cpd_c, cpd_s, cpd_i, cpd_h, cpd_p)

# Check if model is consistent
model.check_model()

True

In [11]:
# Do exact inference using Variable Elimination
infer = VariableElimination(model)

Sanity check: Is P(positive|smoker) = P(positive|non-smoker)?

In [12]:
result = infer.query(['Positive'], evidence={'Smoker': 'yes'})
print("\n\nP(covid | smoker)")
print(result)

result = infer.query(['Positive'], evidence={'Smoker': 'no'})
print("\n\nP(covid | non-smoker)")
print(result)

  0%|          | 0/3 [00:00<?, ?it/s]



P(covid | smoker)
+---------------+-----------------+
| Positive      |   phi(Positive) |
+===============+=================+
| Positive(yes) |          0.1850 |
+---------------+-----------------+
| Positive(no)  |          0.8150 |
+---------------+-----------------+


Eliminating: Covid: 100%|██████████| 3/3 [00:00<00:00, 362.41it/s]



P(covid | non-smoker)
+---------------+-----------------+
| Positive      |   phi(Positive) |
+===============+=================+
| Positive(yes) |          0.1850 |
+---------------+-----------------+
| Positive(no)  |          0.8150 |
+---------------+-----------------+


Confirmed

### Do we get a collider bias?
Is P(positive|smoker,hospital) < P(positive|non-smoker,hospital)?

In [13]:
result = infer.query(['Positive'], evidence={'Smoker': 'yes', 'Hospital': 'yes'})
print("\n\nP(Positive | smoker, hospital)")
print(result)

result = infer.query(['Positive'], evidence={'Smoker': 'no', 'Hospital': 'yes'})
print("\n\nP(Positive | non-smoker, hospital)")
print(result)

Eliminating: Covid: 100%|██████████| 2/2 [00:00<00:00, 447.51it/s]



P(Positive | smoker, hospital)
+---------------+-----------------+
| Positive      |   phi(Positive) |
+===============+=================+
| Positive(yes) |          0.5136 |
+---------------+-----------------+
| Positive(no)  |          0.4864 |
+---------------+-----------------+


P(Positive | non-smoker, hospital)
+---------------+-----------------+
| Positive      |   phi(Positive) |
+===============+=================+
| Positive(yes) |          0.8626 |
+---------------+-----------------+
| Positive(no)  |          0.1374 |
+---------------+-----------------+


In [15]:
result.values

array([0.86262965, 0.13737035])

Yes.

Again, there is a _higher_ chance that you will have Covid-19 if you are a _non-smoker_ in hospital, then if you are a smoker in hospital. 

Probabilities now are different because we have a more detailed model with more parameters, but the basic effect is still there.